In [3]:
import scipy.stats as stats
import pkgutil
import csv
from io import StringIO
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy import stats
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


# these functions should be used to determine if log transformations are needed for the data


def normal_dist_check(input_data):
    """
    Checks for normality in the data by performing a shapiro test.

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        true if the data is normal based on shapiro test, false otherwise.
    """

    data_to_check = input_data.iloc[:, 2:]
    result = stats.shapiro(data_to_check)

    return result[1] < 0.05


# read the test dataset from the package into a dataframe
def read_data_file(
    package_name="metabolomics_analysis_tools",
    file_path="resources/test_dataset/human_cachexia.csv",
):
    if not file_path:
        return "Error: empty input"
    data_bytes = pkgutil.get_data(package_name, file_path)
    if data_bytes is None:
        raise FileNotFoundError("Could not find data file")

    else:
        print("data read successfully")
        data_str = data_bytes.decode("utf-8")
        data_file = StringIO(data_str)
        csv_reader = csv.reader(data_file)
        rows = [row for row in csv_reader]
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df.iloc[:, 2:] = df.iloc[:, 2:].astype(
            {col: float for col in df.iloc[:, 2:].columns}
        )
        print("the shape of the dataframe is: ", df.shape)
        return df
    

# normalization methods
def normalize_by_sum(input_data):
    """
    Normalize the data by dividing each column by the sum of the row

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        pandas.DataFrame: Normalized dataframe
    """
    data_to_process = input_data.iloc[:, 2:]
    normalized_data = data_to_process.div(data_to_process.sum(axis=0), axis=1)
    normed_data = input_data.copy()
    normed_data.iloc[:, 2:] = normalized_data

    return normed_data


def normalize_by_median(input_data):
    """
    Normalize the data by dividing each column by the median of the row

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        pandas.DataFrame: Normalized dataframe
    """

    data_to_process = input_data.iloc[:, 2:]
    normalized_data = data_to_process.div(data_to_process.median(axis=0), axis=1)
    normed_data = input_data.copy()
    normed_data.iloc[:, 2:] = normalized_data

    return normed_data


def normalize_by_reference_sample_PQN(input_data):
    """
    Normalize the data by dividing each column by the PQN of the reference sample.
    PQN has been shown to be effective at normalizing data from different platforms and technologies,
    and can reduce batch effects and other systematic variations in data. However,
    PQN assumes that most genes or proteins are not differentially expressed, which may not be true in all cases.
    Additionally, PQN may not be appropriate for all types of data,
    and other normalization methods may be more appropriate
    depending on the specific research question and experimental design.

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        pandas.DataFrame: Normalized dataframe

    """
    data_to_process = input_data.iloc[:, 2:]
    sample_medians = data_to_process.median(axis=0)
    data_norm = data_to_process.div(sample_medians, axis=1)
    metabolite_means = np.exp(np.log(data_norm).mean(axis=1))
    data_norm = data_norm.div(metabolite_means, axis=0)
    col_medians = data_norm.median(axis=0)
    data_norm = data_norm.div(col_medians, axis=1)
    normed_data = input_data.copy()
    normed_data.iloc[:, 2:] = data_norm

    return normed_data

def data_scaling_mean_centered(normalized_data):
    """
    Scale the data by subtracting the mean of each column from each value in the column

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        pandas.DataFrame: Scaled dataframe
    """

    data_scaled = normalized_data.iloc[:, 2:].subtract(
        normalized_data.mean(axis=0), axis=1
    )
    scaled_data = normalized_data.copy()
    scaled_data.iloc[:, 2:] = data_scaled

    return scaled_data


def data_transformation_log(input_data):
    """
    Transform the data by taking the log10 of each value

    Args:
        input_data (pandas.DataFrame): Dataframe containing the data

    Returns:
        pandas.DataFrame: Transformed dataframe
    """

    data_to_process = input_data.iloc[:, 2:]

    if (data_to_process <= 0).any().any():
        raise ValueError("Error: Non-positive values found in input data")
    data_log10 = data_to_process.apply(np.log10)
    transformed_data = input_data.copy()
    transformed_data.iloc[:, 2:] = data_log10

    return transformed_data


def PCA_analysis(normalized_data, number_of_components=2):
    data_to_process = normalized_data.iloc[:, 2:]
    pca = PCA(n_components=number_of_components)
    principal_components = pca.fit_transform(data_to_process)
    principal_df = pd.DataFrame(
        data=principal_components, columns=[f"PC {i+1}" for i in range(number_of_components)]
    )
    groups = normalized_data.iloc[:, 1]

    # Create a LabelEncoder to encode the groups as integers
    le = LabelEncoder()
    groups_encoded = le.fit_transform(groups)

    # Create a colormap based on the number of unique groups
    cmap = plt.cm.get_cmap('viridis', len(np.unique(groups_encoded)))

    fig, ax = plt.subplots()
    scatter = ax.scatter(
        principal_df["PC 1"], principal_df["PC 2"], c=groups_encoded, cmap=cmap
    )

    # Create a legend with the group labels
    legend_elements = [plt.Line2D([0], [0], marker='o', color=cmap(i), label=label, markersize=7, linestyle='')
                       for i, label in enumerate(le.classes_)]
    legend1 = ax.legend(handles=legend_elements, title="Groups")
    ax.add_artist(legend1)
    ax.set_xlabel("PC 1")
    ax.set_ylabel("PC 2")
    ax.set_title("PCA of Metabolomic Data")

    return fig



def volcano_plot(
    grouped_data,
    sig_threshold=0.05,
    fold_change_threshold=1.5,
    group_name="Muscle loss",
    group_A_name="control",
    group_B_name="cachexic",
):
    """
    Perform a two-sample t-test on each feature and plot the results as a volcano plot
    """

    # convert the data to float
    grouped_data = grouped_data.iloc[:, 1:]
    grouped_data.iloc[:, 1:] = grouped_data.iloc[:, 1:].astype(float)
    # set significance threshold and fold change threshold
    sig_threshold = 0.05
    fold_change_threshold = 1.5

    # create a new DataFrame to store the results
    result_grouped_data = pd.DataFrame(
        columns=["Metabolite", "Fold Change", "P-value", "FDR"]
    )

    # iterate over each feature
    for col in grouped_data.iloc[:, 1:]:
        group_A = grouped_data[grouped_data[group_name] == group_A_name][col]
        group_B = grouped_data[grouped_data[group_name] == group_B_name][col]

        # perform a two-sample t-test for each feature
        t_stat, p_val = stats.ttest_ind(group_A, group_B)

        # calculate the fold change
        mean_A = group_A.mean()
        mean_B = group_B.mean()
        fold_change = mean_B / mean_A

        # correct the p-value for multiple testing using Benjamini-Hochberg procedure
        p_vals = [p_val]
        corrected_p_vals = multipletests(p_vals, method="fdr_bh")[1][0]

        # add the results to the result DataFrame
        result_grouped_data = result_grouped_data.append(
            {
                "Metabolite": col,
                "Fold Change": fold_change,
                "P-value": p_val,
                "FDR": corrected_p_vals,
            },
            ignore_index=True,
        )

    # filter significant features based on FDR and fold change threshold
    sig_features = result_grouped_data[
        (result_grouped_data["FDR"] < sig_threshold)
        & (abs(result_grouped_data["Fold Change"]) > fold_change_threshold)
    ]

    # plot volcano plot
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(
        result_grouped_data["Fold Change"],
        -1 * np.log10(result_grouped_data["P-value"]),
        alpha=0.5,
    )
    ax.scatter(
        sig_features["Fold Change"],
        -1 * np.log10(sig_features["P-value"]),
        color="red",
        alpha=0.5,
    )
    ax.axvline(x=1, color="black", linestyle="--")
    ax.axvline(x=-1, color="black", linestyle="--")
    ax.axhline(y=-np.log10(sig_threshold), color="black", linestyle="--")
    ax.set_xlabel("Fold Change")
    ax.set_ylabel("-Log10 P-value")
    ax.set_title("Volcano Plot")

    return fig



def ma_plot(
    grouped_data,
    group_name="Muscle loss",
    group_A_name="control",
    group_B_name="cachexic",
):
    """
    Create an MA plot
    """
    # Calculate the log-fold change and mean expression
    cachexic_mean = (
        grouped_data[grouped_data[group_name] == group_B_name].iloc[:, 1:].mean()
    )
    control_mean = (
        grouped_data[grouped_data[group_name] == group_A_name].iloc[:, 1:].mean()
    )
    logFC = np.log2(cachexic_mean / control_mean)
    data_mean = np.log2(grouped_data.iloc[:, 1:].mean(axis=0))
    # Create the MA plot
    fig, ax = plt.subplots()
    ax.scatter(data_mean, logFC, s=10, alpha=0.5, color="black")
    ax.axhline(y=0, color="gray", linestyle="--")
    ax.set_xlabel("Mean Expression (log2)")
    ax.set_ylabel("Log-Fold Change (log2)")
    ax.set_title("MA Plot")

    return fig


In [4]:
import PySimpleGUI as sg # assuming that the PySimpleGUI is stored in the myapp folder

# import the functions from the metabolomics_analysis_tools package
from metabolomics_analysis_tools import (
    read_data_file,
    normal_dist_check,
    normalize_by_mean,
    data_scaling_mean_centered,
    data_transformation_log,
    PCA_analysis,
    volcano_plot,
    ma_plot,
)

# define the layout of the GUI


ImportError: cannot import name 'read_data_file' from 'metabolomics_analysis_tools' (/Users/yuzhijian/opt/anaconda3/lib/python3.9/site-packages/metabolomics_analysis_tools/__init__.py)

In [4]:
import PySimpleGUI as sg
import pandas as pd
from scipy import stats



def display_table(df):
    """
    Display a Pandas DataFrame in a PySimpleGUI table.
    """
    header_list = list(df.columns[:5]) # display only first 5 columns
    data = df.iloc[:, :5].values.tolist() # display only first 5 columns
    layout = [
        [sg.Table(values=data, headings=header_list, max_col_width=25,
                  auto_size_columns=True, justification='center',
                  num_rows=min(25, len(data)))],
        [sg.Button('Check Data Quality'), sg.Button('Exit')]
    ]
    window = sg.Window('Table', layout)
    while True:
        event, _ = window.read()
        if event == 'Exit' or event == sg.WIN_CLOSED:
            break
        elif event == 'Check Data Quality':
            # check data quality using the normal_dist_check function
            if normal_dist_check(df):
                sg.popup('The data is normally distributed.')
            else:
                sg.popup('The data is not normally distributed.')
    window.close()

# define the GUI layout
layout = [
    [sg.Text("Select the CSV file to read:")],
    [sg.Input(), sg.FileBrowse()],
    [sg.OK(), sg.Cancel()]
]

# create the window
window = sg.Window("Read CSV File", layout)

# event loop
while True:
    event, values = window.read()
    if event == "OK":
        # read the CSV file using the read_data_file function
        try:
            df = read_data_file(file_path=values[0])
        except FileNotFoundError:
            sg.popup_error("File not found. Please select a valid CSV file.")
            continue
        except Exception as e:
            sg.popup_error(f"An error occurred while reading the file: {str(e)}")
            continue
        
        # display the DataFrame in a table
        display_table(df)
    elif event == "Cancel" or event == sg.WIN_CLOSED:
        break

window.close()


data read successfully
the shape of the dataframe is:  (77, 65)
